In [1]:
using DelimitedFiles
include("utils.jl")

┌ Warning: `getindex(o::PyObject, s::AbstractString)` is deprecated in favor of dot overloading (`getproperty`) so elements should now be accessed as e.g. `o."s"` instead of `o["s"]`.
│   caller = top-level scope at none:0
└ @ Core none:0


load (generic function with 1 method)

In [2]:
using PyCall
using SparseArrays
const scipy_sparse_find = pyimport("scipy.sparse")["find"]

py"""
import numpy as np
import scipy.sparse as sp
import torch
"""

function load()
# TODO: Change for multiple datasets
    # PyCall
    py"""
import numpy as np
import scipy.sparse as sp
import torch


def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in
                    enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)),
                             dtype=np.int32)
    return labels_onehot

def normalize(mx):
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

def load_data(path="cora/", dataset="cora"):
    #Load citation network dataset (cora only for now)
    print('Loading {} dataset...'.format(dataset))

    idx_features_labels = np.genfromtxt("{}{}.content".format(path, dataset),
                                        dtype=np.dtype(str))
    features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)
    labels = encode_onehot(idx_features_labels[:, -1])

    # build graph
    idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
    idx_map = {j: i for i, j in enumerate(idx)}
    edges_unordered = np.genfromtxt("{}{}.cites".format(path, dataset),
                                    dtype=np.int32)
    edges = np.array(list(map(idx_map.get, edges_unordered.flatten())),
                     dtype=np.int32).reshape(edges_unordered.shape)
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(labels.shape[0], labels.shape[0]),
                        dtype=np.float32)

    # build symmetric adjacency matrix
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

    features = normalize(features)
    adj = normalize(adj + sp.eye(adj.shape[0]))

    return adj, np.transpose(features), labels
    """

    adj, features, labels = py"load_data"()
    (I, J, V) = scipy_sparse_find(adj)
    # Zero-indexing issue
    adj = sparse(I .+ 1, J .+ 1, V)

    (I, J, V) = scipy_sparse_find(features)
    # Zero-indexing issue
    features = sparse(I .+ 1, J .+ 1, V)

    # TODO: Uncomment the following
    # Normalize feature
    # features = features./sum(features,2)
    # Add identity matrix
    #adj += sparse(I, size(adj,1), size(adj,2))
    # Normalize
    #adj = adj./sum(adj,2)

    # Indexes
    idx_train = 1:140
    idx_val = 141:500
    idx_test = 501:size(features,2)

    return adj, features, labels, idx_train, idx_val, idx_test
end

┌ Warning: `getindex(o::PyObject, s::AbstractString)` is deprecated in favor of dot overloading (`getproperty`) so elements should now be accessed as e.g. `o."s"` instead of `o["s"]`.
│   caller = top-level scope at In[2]:3
└ @ Core In[2]:3


load (generic function with 1 method)

In [64]:
using Knet: dropout, nll, Data
using Statistics
include("layers.jl")

# TODO: Can convert to chain structure
struct GCN
    layer1::GraphConvolution
    layer2::GraphConvolution
    pdrop
end

function GCN(nfeat::Int, nhid::Int, nclass::Int, pdrop=0)
    GCN(GraphConvolution(nfeat, nhid, relu), GraphConvolution(nhid, nclass, identity), pdrop)
end

function (c::GCN)(x, adj)
    x = c.layer1(x, adj)
    x = dropout(x, c.pdrop)
    x = c.layer2(x, adj)
    #return logp(x, dims=1)
    return x
end

(c::GCN)(x, adj, y) = nll(c(x, adj),y)

# TODO: Data should include adj
function (c::GCN)(d::Data)
    mean(c(x, adj, y) for (x, adj, y) in d)
end

┌ Warning: `getindex(o::PyObject, s::AbstractString)` is deprecated in favor of dot overloading (`getproperty`) so elements should now be accessed as e.g. `o."s"` instead of `o["s"]`.
│   caller = top-level scope at none:0
└ @ Core none:0


In [4]:
using Knet: Adam

# Load data
adj, features, labels, idx_train, idx_val, idx_test = load()

# TODO: take user inputs
struct args
    epochs
    lr
    weight_decay
    hidden
    pdrop
end
arguments = args(200, 0.01, 5e-4, 16, 0.5)

# Model and optimizer
model = GCN(size(features,1),
            arguments.hidden,
            # TODO: Change nclass
            size(labels,2),
            #nclass=max(labels) + 1,
            arguments.pdrop)

dtrn = features[idx_train]
atrn = adj[idx_train]
dval = features[idx_val]
aval = adj[idx_val]
dtst = features[idx_test]
atst = adj[idx_test]

2208-element SparseVector{Float64,Int64} with 1 stored entry:
  [45  ]  =  0.2

In [5]:
using Pkg;
for p in ["Knet", "Plots"]
    if !haskey(Pkg.installed(),p)
        Pkg.add(p);
    end
end

In [6]:
Pkg.add("IterTools")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 % Resolving package versions...
  Updating `~/.julia/environments/v1.1/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.1/Manifest.toml`
 [no changes]


In [7]:
using IterTools: ncycle, takenth

In [8]:
using Base.Iterators: flatten

In [9]:
training = adam(model, ncycle(dtrn, arguments.epochs), lr=arguments.lr)

Knet.Minimize{IterTools.NCycle{SparseVector{Float32,Int64}}}(IterTools.NCycle{SparseVector{Float32,Int64}}(  [119]  =  0.05
  [126]  =  0.05, 200), GCN(GraphConvolution(P(Array{Float32,2}(16,1433)), P(Array{Float32,1}(16)), NNlib.relu), GraphConvolution(P(Array{Float32,2}(7,16)), P(Array{Float32,1}(7)), identity), 0.5), Adam(0.01, 0.9, 0.999, 1.0e-8, 0, 0.0, nothing, nothing), nothing)

In [70]:
include("layers.jl")

┌ Warning: `getindex(o::PyObject, s::AbstractString)` is deprecated in favor of dot overloading (`getproperty`) so elements should now be accessed as e.g. `o."s"` instead of `o["s"]`.
│   caller = top-level scope at none:0
└ @ Core none:0


In [71]:
size(features)

(1433, 2708)

In [72]:
size(adj)

(2708, 2708)

In [73]:
using Knet: logp

In [76]:
output = model(features, adj)

(16, 2708)(7, 2708)

7×2708 SparseMatrixCSC{Float64,Int64} with 18956 stored entries:
  [1   ,    1]  =  -0.0048461
  [2   ,    1]  =  0.000309735
  [3   ,    1]  =  -0.00425457
  [4   ,    1]  =  0.000821694
  [5   ,    1]  =  0.00611521
  [6   ,    1]  =  0.00218222
  [7   ,    1]  =  -0.000294031
  [1   ,    2]  =  -0.00310794
  [2   ,    2]  =  -0.000488848
  [3   ,    2]  =  -0.000876225
  [4   ,    2]  =  0.000385584
  [5   ,    2]  =  0.00353407
  ⋮
  [3   , 2707]  =  -0.000860082
  [4   , 2707]  =  0.00225941
  [5   , 2707]  =  -0.00382417
  [6   , 2707]  =  0.000982476
  [7   , 2707]  =  8.99052e-5
  [1   , 2708]  =  -0.00695635
  [2   , 2708]  =  -0.0158236
  [3   , 2708]  =  -0.00573691
  [4   , 2708]  =  0.00146955
  [5   , 2708]  =  0.00487449
  [6   , 2708]  =  0.00302706
  [7   , 2708]  =  0.0059386

In [78]:
labels2 = mapslices(argmax, labels ,dims=2)

2708×1 Array{Int64,2}:
 6
 7
 4
 4
 5
 5
 2
 6
 6
 2
 6
 3
 5
 ⋮
 2
 5
 5
 5
 5
 6
 3
 3
 3
 3
 1
 6

In [99]:
output3 = convert(Array{Float64,2}, output)  

7×2708 Array{Float64,2}:
 -0.0048461    -0.00310794   -0.00340505   …   0.00665658   -0.00695635
  0.000309735  -0.000488848  -0.00382896       0.00638235   -0.0158236 
 -0.00425457   -0.000876225   0.00203873      -0.000860082  -0.00573691
  0.000821694   0.000385584   0.00717931       0.00225941    0.00146955
  0.00611521    0.00353407    0.000656041     -0.00382417    0.00487449
  0.00218222    0.00317702   -0.00109364   …   0.000982476   0.00302706
 -0.000294031  -0.00339911   -0.00299544       8.99052e-5    0.0059386 

In [110]:
output3[:,idx_train]

7×140 Array{Float64,2}:
 -0.0048461    -0.00310794   -0.00340505   …   0.00259081    0.00323576 
  0.000309735  -0.000488848  -0.00382896      -0.000894565  -0.000640614
 -0.00425457   -0.000876225   0.00203873      -0.0111251     0.00473097 
  0.000821694   0.000385584   0.00717931      -0.00501453    0.00140762 
  0.00611521    0.00353407    0.000656041     -0.00336287    0.00117353 
  0.00218222    0.00317702   -0.00109364   …  -0.00458765    0.000515495
 -0.000294031  -0.00339911   -0.00299544       0.00054167   -0.00358892 

In [111]:
nll(output3[:,idx_train], labels2[ idx_train] )

1.9446427005415956

In [113]:
using Knet: accuracy

In [114]:
accuracy(output3[:,idx_train], labels2[ idx_train] )

0.2